***GENERATED CODE FOR creditcardfrauddetectionclassification PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run creditcardfrauddetectionclassificationHooks.ipynb
try:
	#sourcePreExecutionHook()

	creditcardfrauddetectionclassificationhdfs = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/extra/tempFile1676284559.csv', 'filename': 'tempFile1676284559.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(creditcardfrauddetectionclassificationhdfs)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run creditcardfrauddetectionclassificationHooks.ipynb
try:
	#transformationPreExecutionHook()

	creditcardfrauddetectionclassificationautofe = TransformationMain.run(creditcardfrauddetectionclassificationhdfs,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[Time]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "487.48", "stddev": "423.48", "min": "123", "max": "1657", "missing": "0"}, "updatedLabel": "creditcardTrainData[Time]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V1]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.45", "stddev": "1.39", "min": "-5.852465108", "max": "1.586092522", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V1]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V2]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.41", "stddev": "1.14", "min": "-5.931717441", "max": "5.26737615", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V2]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V3]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.76", "stddev": "0.96", "min": "-3.597858245", "max": "3.772856852", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V3]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V4]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.11", "stddev": "1.28", "min": "-4.43421128", "max": "3.997905588", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V4]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V5]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.05", "stddev": "1.02", "min": "-3.125908514", "max": "7.672543966", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V5]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V6]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.01", "stddev": "1.23", "min": "-3.498447175", "max": "4.885995928", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V6]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V7]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.01", "stddev": "0.78", "min": "-4.925568043", "max": "4.185906257", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V7]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V8]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.05", "stddev": "1.06", "min": "-7.494657608", "max": "1.875254268", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V8]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V9]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.05", "stddev": "0.84", "min": "-2.770089277", "max": "5.436633396", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V9]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V10]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.04", "stddev": "0.95", "min": "-2.772272145", "max": "8.737457806", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V10]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V11]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.12", "stddev": "1.0", "min": "-2.437157248", "max": "3.202033207", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V11]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V12]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.33", "stddev": "0.7", "min": "-2.899907388", "max": "2.152055116", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V12]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V13]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.14", "stddev": "0.97", "min": "-2.656600591", "max": "3.078366245", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V13]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V14]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.15", "stddev": "0.87", "min": "-6.576788553", "max": "1.642575038", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V14]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V15]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.14", "stddev": "0.92", "min": "-3.618059825", "max": "2.501394702", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V15]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V16]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.08", "stddev": "0.74", "min": "-3.133093384", "max": "1.514765448", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V16]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V17]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.08", "stddev": "0.76", "min": "-5.400014373", "max": "3.006908567", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V17]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V18]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.11", "stddev": "0.71", "min": "-2.76226609", "max": "2.606244472", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V18]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V19]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.05", "stddev": "0.76", "min": "-3.463993232", "max": "2.862706058", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V19]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V20]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.09", "stddev": "0.49", "min": "-2.538941486", "max": "3.564137979", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V20]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V21]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.05", "stddev": "0.83", "min": "-4.134607658", "max": "7.53197739", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V21]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V22]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.16", "stddev": "0.6", "min": "-2.788601452", "max": "1.461535075", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V22]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V23]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.04", "stddev": "0.32", "min": "-1.786328942", "max": "3.150413499", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V23]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V24]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.01", "stddev": "0.63", "min": "-1.867208192", "max": "1.108439997", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V24]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V25]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.1", "stddev": "0.41", "min": "-1.389079168", "max": "1.125918008", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V25]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V26]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.04", "stddev": "0.42", "min": "-0.984010664", "max": "1.286200506", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V26]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V27]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.07", "stddev": "0.29", "min": "-1.346577772", "max": "2.468867101", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V27]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[V28]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.01", "stddev": "0.26", "min": "-2.733887119", "max": "1.573083584", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[V28]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[Amount]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "43.34", "stddev": "101.16", "min": "0.0", "max": "881.13", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creditcardTrainData[Amoun..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creditcardTrainData[Class]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.06", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "creditcardTrainData[Class..."}]}))

	#transformationPostExecutionHook(creditcardfrauddetectionclassificationautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run creditcardfrauddetectionclassificationHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(creditcardfrauddetectionclassificationautofe, ["creditcardTrainData[Time]", "creditcardTrainData[V1]", "creditcardTrainData[V2]", "creditcardTrainData[V3]", "creditcardTrainData[V4]", "creditcardTrainData[V5]", "creditcardTrainData[V6]", "creditcardTrainData[V7]", "creditcardTrainData[V8]", "creditcardTrainData[V9]", "creditcardTrainData[V10]", "creditcardTrainData[V11]", "creditcardTrainData[V12]", "creditcardTrainData[V13]", "creditcardTrainData[V14]", "creditcardTrainData[V15]", "creditcardTrainData[V16]", "creditcardTrainData[V17]", "creditcardTrainData[V18]", "creditcardTrainData[V19]", "creditcardTrainData[V20]", "creditcardTrainData[V21]", "creditcardTrainData[V22]", "creditcardTrainData[V23]", "creditcardTrainData[V24]", "creditcardTrainData[V25]", "creditcardTrainData[V26]", "creditcardTrainData[V27]", "creditcardTrainData[V28]", "creditcardTrainData[Amount]"], "creditcardTrainData[Class]")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

